В этом ноутбуке мы займемся проверкой на наличие прямых упоминаний ПАВ

In [1]:
import pandas as pd
import stanza
import tqdm as notebook_tqdm
import nltk
from nltk.corpus import stopwords
import string
import numpy as np

d:\Python\course work\Yandex Music\2017-2024 txts\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# подгружаем файл с лемматизированными текстами
df_final=pd.read_parquet(r'D:\Python\course work\Yandex Music\2017-2024 txts\df_final_lemmatized.parquet')
df_final.head()

,Исполнитель,Трек,Собран,Жанр,Доп. информация,Фит,Совместный релиз,Альбом,Год выхода,Текст трека,Языки в тексте,Части текста,Обработанные части,Очищенные леммы
original_index,,,,,,,,,,,,,,
https://music.yandex.ru/album/17778130/track/53360237,Pavel Kempel,Русская весна,1.0,русский рэп,Нет,Нет,Несовместный релиз,Буква,2019.0,"Серые тучи, серые люди\nСерые кучи, серые будн...",[ru],"[Серые тучи, серые люди\nСерые кучи, серые буд...","[[серый, туча, ,, серый, человек, серый, куча,...","[[серый, туча, серый, человек, серый, куча, се..."
https://music.yandex.ru/album/6931563/track/50255124,The Simba,Карусель,1.0,танцевальная музыка,The Simba Remix,Нет,M'Dee,Сингл,2019.0,"Ты поздно поняла, поняла, поняла — я не такой\...",[ru],"[Ты поздно поняла, поняла, поняла — я не такой...","[[ты, поздно, понять, ,, понять, ,, понять, —,...","[[поздно, понять, понять, понять, лгать, ждать..."
https://music.yandex.ru/album/9521959/track/61168371,Кузьма,Балдёж,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"День и ночь балдеж, Россию не поймешь\nТанцуй ...","[ru, bg]","[День и ночь балдеж, Россию не поймешь\nТанцуй...","[[день, и, ночь, балдеж, ,, Россия, не, понять...","[[день, ночь, балдеж, Россия, понять, таныть, ..."
https://music.yandex.ru/album/21509172/track/101776208,SaNni,Где бывают боги,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Это зачем?\nЧтоб боялись\nКого боятся, тот и г...","[ru, es, sr]","[Это зачем?\nЧтоб боялись\nКого боятся, тот и ...","[[это, зачем, ?, чтобы, бояться, кто, бояться,...","[[это, бояться, бояться, главний, слушать, схо..."
https://music.yandex.ru/album/21509188/track/101776286,SaNni,Ламбада - Румба,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Ламбада, румба, крути панорама\nМама хасла, пр...",[ru],"[Ламбада, румба, крути панорама\nМама хасла, п...","[[ламбада, ,, румба, ,, крутить, панорама, мам...","[[ламбада, румба, крутить, панорама, мама, хас..."


In [4]:
dictionary=pd.read_excel(r'D:\Python\course work\Yandex Music\2017-2024 txts\drug_dictionary (version 1).xlsx')
dictionary

,Управление ООН по наркотикам и преступности,Управление ООН по наркотикам и преступности (англ),Постановление Правительства РФ,Список наркотических веществ находящихся под международным контролем,Список наркотических веществ находящихся под международным контролем (англ)
0,Aцеторфин,Acetorphine,N-(адамантан-1-ил)-1-бензил-1Н-индазол-3-карбо...,АКРИЛОИЛФЕНТАНИЛ,Abalgin
1,Ацеторфина гидрохлорид,Acetorphine hydrochloride,N-(адамантан-1-ил)-1-бензил-1H-индол-3-карбокс...,АЛЛИЛПРОДИН,Abcari
2,Ацетил-альфа-метилфентанил,Acetyl-alpha-methylfentanyl,N-(адамантан-1-ил)-1-бутил-1H-индазол-3-карбок...,АЛЬФАМЕПРОДИН,Abkari
3,Ацетилдигидрокодеин,Acetyldihydrocodeine,N-(адамантан-1-ил)-1-бутил-1H-индол-3-карбоксамид,АЛЬФАМЕТАДОЛ,Abhini
4,Ацетилдигидрокодеина гидрохлорид,Acetyldihydrocodeine hydrochloride,N-(адамантан-1-ил)-1-бутил-4-метил-5-фенил-1H-...,АЛЬФА-МЕТИЛТИОФЕНТАНИЛ,Abitran
...,...,...,...,...,...
2271,NaN,NaN,NaN,NaN,Zefalgin
2272,NaN,NaN,NaN,NaN,Zeller
2273,NaN,NaN,NaN,NaN,Zeropyn
2274,NaN,NaN,NaN,NaN,Zideron


In [27]:
[x.lower().strip() for x in dictionary[dictionary['Управление ООН по наркотикам и преступности'].notna()]['Управление ООН по наркотикам и преступности'].to_list()]

['aцеторфин',
 'ацеторфина гидрохлорид',
 'ацетил-альфа-метилфентанил',
 'ацетилдигидрокодеин',
 'ацетилдигидрокодеина гидрохлорид',
 'ацетилметадол',
 'альфентанил',
 'альфентанила гидрохлорид',
 'аллобарбитал',
 'аллобарбитал-аминофеназон',
 'аллилпродин',
 'аллилпродина гидрохлорид',
 'альфацетилметадол',
 'альфацетилметадола гидрохлорид',
 'альфамепродин',
 'альфаметадол',
 'альфа-метилфентанил',
 'альфа-метилфентанила гидрохлорид',
 'альфа-метилтиофентанил',
 'альфа-метилтиофентанила гидрохлорид',
 'альфапродин',
 'альфапродина гидрохлорид',
 'альпразолам',
 'амфепрамон',
 'амфепрамона глютамат',
 'амфепрамона гидрохлорид',
 'амфепрамона резинат',
 'амфетамин',
 'амфетамина ацетилсалицилат',
 'амфетамина адипат',
 'амфетамина п-аминофенилацетат',
 'амфетамина аспартат',
 'амфетамина ацетат',
 'амфетамина гидрохлорид',
 'амфетамина пентобарбитурат',
 'амфетамина фосфат',
 'амфетамина резинат',
 'амфетамина сульфат',
 'амфетамина таннат',
 'амфетамина тартрат',
 'аминептин',
 'амине

In [28]:
dictionary_en_1=[x.lower().strip() for x in dictionary[dictionary['Управление ООН по наркотикам и преступности (англ)'].notna()]['Управление ООН по наркотикам и преступности (англ)'].to_list()]
dictionary_en_2=[x.lower().strip() for x in dictionary[dictionary['Список наркотических веществ находящихся под международным контролем (англ)'].notna()]['Список наркотических веществ находящихся под международным контролем (англ)'].to_list()]
dictionary_ru_1=[x.lower().strip() for x in dictionary[dictionary['Управление ООН по наркотикам и преступности'].notna()]['Управление ООН по наркотикам и преступности'].to_list()]
dictionary_ru_2=[x.lower().strip() for x in dictionary[dictionary['Постановление Правительства РФ'].notna()]['Постановление Правительства РФ'].to_list()]
dictionary_ru_3=[x.lower().strip() for x in dictionary[dictionary['Список наркотических веществ находящихся под международным контролем'].notna()]['Список наркотических веществ находящихся под международным контролем'].to_list()]

Теперь надо все эти списки провести по пайплайну лемматизации, токенизации и прочего и затем объединить

In [31]:
dictionary_ru_full = dictionary_ru_1 + dictionary_ru_2 + dictionary_ru_3
dictionary_en_full=dictionary_en_1+dictionary_en_2

In [ ]:
# Инициализация пайплайна для русского языка
nlp_ru = stanza.Pipeline('ru', processors='tokenize,pos,lemma', use_gpu=False, verbose=False)

# Уникальные слова
dictionary_ru_unique = list(set(dictionary_ru_full))

# Лемматизированные слова
lemmatized_dict_ru = []

for word in dictionary_ru_unique:
    doc = nlp_ru(word)
    for sentence in doc.sentences:
        for token in sentence.words:
            lemmatized_dict_ru.append(token.lemma.lower())  # Приводим к нижнему регистру

In [ ]:
# Инициализация пайплайна для английского языка
nlp_en = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma', use_gpu=False, verbose=False)

# Уникальные слова из списка
dictionary_en_unique = list(set(dictionary_en_full))

# Лемматизированный словарь
lemmatized_dict_en = []

for word in dictionary_en_unique:
    doc = nlp_en(word)
    for sentence in doc.sentences:
        for token in sentence.words:
            lemmatized_dict_en.append(token.lemma.lower())

In [59]:
# Загрузим списки, если еще не загружены
nltk.download('stopwords')

# Стоп-слова и пунктуация
stopwords_ru = set(stopwords.words('russian'))
stopwords_en = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Удаление стоп-слов, пунктуации и цифр — русский
lemmatized_dict_ru_filtered = [
    lemma for lemma in lemmatized_dict_ru
    if lemma not in stopwords_ru and
       lemma not in punctuation and
       not lemma.isdigit() and
       lemma.strip()
]

# Удаление стоп-слов, пунктуации и цифр — английский
lemmatized_dict_en_filtered = [
    lemma for lemma in lemmatized_dict_en
    if lemma not in stopwords_en and
       lemma not in punctuation and
       not lemma.isdigit() and
       lemma.strip()
]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alex-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
# Преобразуем списки в множества для быстрой проверки вхождения
set_ru = set(lemmatized_dict_ru_filtered)
set_en = set(lemmatized_dict_en_filtered)

In [61]:
def find_forbidden_lemmas(lemmas_by_paragraph, set_ru, set_en):
    # Объединяем все леммы из всех абзацев
    all_lemmas = [lemma for paragraph in lemmas_by_paragraph for lemma in paragraph.tolist()]
    # Находим пересечения
    found_ru = set_ru.intersection(all_lemmas)
    found_en = set_en.intersection(all_lemmas)
    found_total = sorted(found_ru.union(found_en))
    return found_total

In [62]:
df_final['Найденные запрещённые леммы'] = df_final['Очищенные леммы'].apply(
    lambda x: find_forbidden_lemmas(x, set_ru, set_en)
)

df_final['Есть запрещённые леммы'] = df_final['Найденные запрещённые леммы'].apply(
    lambda x: bool(x)
)

In [63]:
df_final

,Исполнитель,Трек,Собран,Жанр,Доп. информация,Фит,Совместный релиз,Альбом,Год выхода,Текст трека,Языки в тексте,Части текста,Обработанные части,Очищенные леммы,Есть запрещённые леммы,Найденные запрещённые леммы
original_index,,,,,,,,,,,,,,,,
https://music.yandex.ru/album/17778130/track/53360237,Pavel Kempel,Русская весна,1.0,русский рэп,Нет,Нет,Несовместный релиз,Буква,2019.0,"Серые тучи, серые люди\nСерые кучи, серые будн...",[ru],"[Серые тучи, серые люди\nСерые кучи, серые буд...","[[серый, туча, ,, серый, человек, серый, куча,...","[[серый, туча, серый, человек, серый, куча, се...",True,[дом]
https://music.yandex.ru/album/6931563/track/50255124,The Simba,Карусель,1.0,танцевальная музыка,The Simba Remix,Нет,M'Dee,Сингл,2019.0,"Ты поздно поняла, поняла, поняла — я не такой\...",[ru],"[Ты поздно поняла, поняла, поняла — я не такой...","[[ты, поздно, понять, ,, понять, ,, понять, —,...","[[поздно, понять, понять, понять, лгать, ждать...",True,[baby]
https://music.yandex.ru/album/9521959/track/61168371,Кузьма,Балдёж,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"День и ночь балдеж, Россию не поймешь\nТанцуй ...","[ru, bg]","[День и ночь балдеж, Россию не поймешь\nТанцуй...","[[день, и, ночь, балдеж, ,, Россия, не, понять...","[[день, ночь, балдеж, Россия, понять, таныть, ...",False,[]
https://music.yandex.ru/album/21509172/track/101776208,SaNni,Где бывают боги,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Это зачем?\nЧтоб боялись\nКого боятся, тот и г...","[ru, es, sr]","[Это зачем?\nЧтоб боялись\nКого боятся, тот и ...","[[это, зачем, ?, чтобы, бояться, кто, бояться,...","[[это, бояться, бояться, главний, слушать, схо...",False,[]
https://music.yandex.ru/album/21509188/track/101776286,SaNni,Ламбада - Румба,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2019.0,"Ламбада, румба, крути панорама\nМама хасла, пр...",[ru],"[Ламбада, румба, крути панорама\nМама хасла, п...","[[ламбада, ,, румба, ,, крутить, панорама, мам...","[[ламбада, румба, крутить, панорама, мама, хас...",False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://music.yandex.ru/album/11864118/track/70310541,XANSY,Тактический дробовик,1.0,рэп и хип-хоп,Нет,Нет,Несовместный релиз,Повесть временных треп,2020.0,вроде\n\nон пытается быть лучше\nтут старания ...,[ru],"[вроде, он пытается быть лучше\nтут старания н...","[[], [он, пытаться, быть, хороший, тут, старан...","[[], [пытаться, хороший, старание, напрасный, ...",False,[]
https://music.yandex.ru/album/34612910/track/134415673,dayerteq,КОНТРА ТИТИ,1.0,русский рэп,Нет,Нет,Несовместный релиз,Сингл,2024.0,дрожь по телу я дебил\nскорострел и чайлдфри\n...,"[ru, bg]",[дрожь по телу я дебил\nскорострел и чайлдфри\...,"[[дрожь, по, тело, я, дебил, скорострел, и, ча...","[[дрожь, тело, дебил, скорострел, чайлдфа, уме...",True,[масло]
https://music.yandex.ru/album/32260658/track/128530219,COLDAH,МИЗАНТРОП,1.0,русский рэп,Нет,"COLDAH, Supa sKKinny, Yaprada, АНДЕРDOGS",Несовместный релиз,ЭМОБЫДЛО,2021.0,"хо-хочуспать, ща как ёбнет бас\n\nКидай нюдсы,...",[ru],"[хо-хочуспать, ща как ёбнет бас, Кидай нюдсы, ...","[[хо-хочуспать, ,, ща, как, быть, бас], [кидат...","[[хо-хочуспать, ща, бас], [кидать, нюдса, кида...",True,"[мефедрон, пара]"


In [ ]:
found_list = df_final['Найденные запрещённые леммы'].to_list()

# Плоский список всех найденных запрещённых лемм
found_list_flattened = [lemma for sublist in found_list for lemma in sublist]

# Только уникальные леммы, отсортированные
unique_lemmas = sorted(set(found_list_flattened))

# Результат
unique_lemmas #наш список, с уникальными леммами обнаруженными в текстах

["'s",
 '-(',
 '1,2',
 '1-',
 '1s',
 '2-',
 '2.',
 '3,4',
 '3-',
 '4-',
 '5-',
 '7-',
 '9-',
 'acid',
 'ag',
 'ah',
 'alpha',
 'asa',
 'b',
 'baby',
 'ban',
 'base',
 'benefit',
 'beta',
 'c',
 'cannabis',
 'cc',
 'cd',
 'chem',
 'china',
 'clear',
 'co',
 'coca',
 'cocaine',
 'cod',
 'codein',
 'codeine',
 'compound',
 'concentrate',
 'cs',
 'cum',
 'd-',
 'dc',
 'df',
 'dmt',
 'dolo',
 'dom',
 'dp',
 'dx',
 'e',
 'ed',
 'end',
 'ergo',
 'eroin',
 'ex',
 'fast',
 'gem',
 'gram',
 'guard',
 'h',
 'heroin',
 'ho',
 'hp',
 'improve',
 'ir',
 'iron',
 'j',
 'jb',
 'l',
 'leaf',
 'long',
 'm-',
 'mda',
 'mem',
 'mit',
 'mix',
 'mono',
 'morphine',
 'mos',
 'mr',
 'ms',
 'n',
 "n'",
 'n-',
 'na',
 'neo',
 'o-',
 'opium',
 'opt',
 'oral',
 'oxy',
 'oxycodone',
 'oн',
 'p',
 'pain',
 'para',
 'paste',
 'percocet',
 'peter',
 'plan',
 'plus',
 'pm',
 'poppy',
 'prepare',
 'promethazine',
 'pu',
 'r',
 'raf',
 'retard',
 'sirop',
 'straw',
 'tan',
 'thousand',
 'tussin',
 'u-',
 'ultra',
 'um',

In [ ]:
unique_lemmas_new=unique_lemmas #наш список, с уникальными леммами обнаруженными в текстах
remove_words = ["'s", '-(', '1,2', '1-', '1s', '2-', '2.', '3,4', '3-', '4-', '5-', '7-', '9-','ag', 'ah','asa', 'b', 'baby','ban', 'base', 'benefit','c','cc', 'cd', 'chem', 'china', 'clear', 'co',
                'cod','compound','cs', 'cum', 'd-', 'dc', 'df', 'dmt', 'dolo','dp', 'dx', 'e', 'ed', 'end', 'ergo','ex', 'fast', 'gem', 'gram', 'guard', 'h','ho', 'hp', 'improve', 'ir', 'iron', 'j',
                'jb', 'l', 'leaf', 'long', 'm-','mix','mono','mos', 'mr', 'ms', 'n', "n'", 'n-', 'na', 'neo', 'o-','oral', 'oxy','oн', 'p', 'pain', 'para', 'paste','peter', 'plan', 'plus', 'pm',
                'prepare','pu', 'r', 'raf', 'retard','straw', 'tan', 'thousand','u-', 'ultra', 'um', 'white', 'would', 'xp','бензо', 'бета', 'биб','вид','железо', 'и(', 'изготовить', 'ил', 'кальций',
                'курение','лист','масло','медицинский','медь','млечный','мот','н-лат', 'настойка', 'натрий','нафталина','никель', 'нитрат', 'оксид','ол','пара','препарат','продукт','производная',
                'промежуточный', 'пропан', 'пропирать','разный','смесь', 'смола', 'сок', 'солома','уксусный','фенол','хлор', 'экстракт', 'этил','эфир', 'дом', 'кока', 'маковый', 'кислота','aльфа',
                'мид', 'мак', 'дэт', 'дох', 'beta', 'poppy', 'acid', 'мда', 'магний', 'концентрат', 'альфа', 'борат', 'coca', 'бутан', 'alpha', 'метил', 'дма', 'opt', 'eroin', 'concentrate',
                'пент', 'mit', 'mem'] # почистим от мусора

# Удаляем слова из remove_words
filtered_list = [word for word in unique_lemmas_new if word not in remove_words]
filtered_set=set(filtered_list)

Почистили от потенциально не подходящих слов и сейчас еще раз проверим датафрейм

In [242]:
# Функция для обработки одной строки
def find_mentions(lemma_arrays, filtered_set):
    # Объединяем массивы в плоский список
    all_lemmas = []
    for arr in lemma_arrays:
        all_lemmas.extend(arr.tolist())  # Преобразуем numpy array -> list и добавляем

    # Ищем пересечение
    found = set(all_lemmas) & filtered_set

    if found:
        return True, list(found)
    else:
        return False, []

# Применяем к DataFrame
df_final['Есть упоминание'], df_final['Обнаруженное упоминание'] = zip(*df_final['Очищенные леммы'].apply(lambda x: find_mentions(x, filtered_set)))

In [245]:
direct_mention_tracks=df_final[df_final['Есть упоминание']==True]
direct_mention_tracks

,Исполнитель,Трек,Собран,Жанр,Доп. информация,Фит,Совместный релиз,Альбом,Год выхода,Текст трека,Языки в тексте,Части текста,Обработанные части,Очищенные леммы,Есть упоминание,Обнаруженное упоминание
original_index,,,,,,,,,,,,,,,,
https://music.yandex.ru/album/4654989/track/36879875,Kuznetsky Squad,Наркодилер,1.0,русский рэп,Нет,Нет,Несовместный релиз,Кузнецкий Тейп,2017.0,Эй\n\nДьяволам вокруг не нужно слышать правду\...,[ru],"[Эй, Дьяволам вокруг не нужно слышать правду\n...","[[эй], [дьявол, вокруг, не, нужный, слышать, п...","[[эй], [дьявол, вокруг, нужный, слышать, правд...",True,[опиум]
https://music.yandex.ru/album/15286876/track/62324223,Заги Бок,В книге всё было по-другому,1.0,русский рэп,4 раунд 17ib,Нет,Несовместный релиз,17 независимый баттл,2020.0,"Да, блять, ха-ха-ха-ха, кхм-кхм\nПау, бля, let...","[ru, es]","[Да, блять, ха-ха-ха-ха, кхм-кхм\nПау, бля, le...","[[], [да, кто, вообще, видеть, первоисточник, ...","[[], [вообще, видеть, первоисточник, знать, од...",True,[каннабис]
https://music.yandex.ru/album/12712030/track/73309005,ЗАМАЙ,Алкоголь,1.0,русский рэп,Нет,Нет,Несовместный релиз,Андрей,2020.0,"Угу, всегда не против, на весу не наливать\nНу...","[ru, es]","[Угу, всегда не против, на весу не наливать\nН...","[[угу, ,, всегда, не, против, ,, на, вес, не, ...","[[угу, против, вес, наливать, знать, это, плас...",True,[кокаин]
https://music.yandex.ru/album/8768150/track/58009828,ЗАМАЙ,Вершина,1.0,русский рэп,Нет,Нет,Несовместный релиз,Ричард 3,2019.0,Я на вершине мира на тринадцатом этаже\nMarrio...,[ru],[Я на вершине мира на тринадцатом этаже\nMarri...,"[[я, на, вершина, мир, на, тринадцатый, этаж, ...","[[вершина, мир, тринадцатый, этаж, Marriott, В...",True,[гашиша]
https://music.yandex.ru/album/4133159/track/33775078,ЗАМАЙ,Новый,1.0,русский рэп,Нет,Нет,Несовместный релиз,Из замка в замок,2017.0,Новых тем и новых дев\nНовый берег в новый бег...,[ru],[Новых тем и новых дев\nНовый берег в новый бе...,"[[новый, тема, и, новый, дев, новый, берег, в,...","[[новый, тема, новый, дев, новый, берег, новый...",True,[гашиша]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://music.yandex.ru/album/5988732/track/44620416,Слава КПСС,На мерзком,1.0,русский рэп,Нет,Нет,ЗАМАЙ,ОВЕРХАЙП 3,2018.0,"Я на мерзком, на мерзком, эй\nПослушайте как я...",[ru],"[Я на мерзком, на мерзком, эй\nПослушайте как ...","[[я, на, мерзкий, ,, на, мерзкий, ,, эй, послу...","[[мерзкий, мерзкий, эй, послушать, раздавать, ...",True,[кокаин]
https://music.yandex.ru/album/25073133/track/39439324,Ameriqa,Старт,1.0,русский рэп,Нет,Нет,May Wave$,Surfin'2,2018.0,"Я только встал, чтобы сделать дела\nПоловина д...","[en, ru]","[Я только встал, чтобы сделать дела\nПоловина ...","[[я, только, встать, ,, чтобы, делать, дело, п...","[[встать, делать, дело, половина, дури, мешать...",True,[percocet]
https://music.yandex.ru/album/29973344/track/123186085,dekma,СПИДРАН / ГИЛЬЗЫ,1.0,рэп и хип-хоп,Нет,Нет,"Umsy, Warykid",МЕТРОН,2021.0,"Я, как слушатель трепа от мира политики\nПокур...",[ru],"[Я, как слушатель трепа от мира политики\nПоку...","[[я, ,, как, слушатель, треп, от, мир, политик...","[[слушатель, треп, мир, политика, покурить, га...",True,[дезоморфин]


In [246]:
direct_mention_tracks.to_parquet('direct_mention_tracks.parquet')

УРА МЫ ВЫЯСНИЛИ ГДЕ ПРЯМЫЕ УПОМИНАНИЯ!!

In [103]:
#df_final[df_final['Найденные запрещённые леммы'].apply(lambda x: len(x) > 0)]